# `null`과 `Optional`

자바에서 참조 타입의 값은 `null`을 포함하며 참조할 어떤 객체(object)도 없음을 의미한다.

`null` 참조에 대해 메소드를 호출하려 하면 `NullPointerException`이 발생한다.

초보 자바 프로그래머의 실수로 `NullPointerException` 발생시키며 프로그램이 종료되는 일은 흔히 일어나곤 한다.

`null`을 역참조(dereference, 참조 타입의 값에 해당하는 객체의 내용을 가져오기)하는 것을 피하는 방법은 크게 두 가지로 나누어 생각할 수 있다.
1. 참조 타입의 값을 읽을 때마다 (메소드 호출도 여기에 포함) 매번 `null`인지 검사하여 처리하기
2. `null`로 초기화하거나 대입하지 않도록 방어하기

첫째 방법, 그러니깨 매번 읽을 때마다 검사하는 방법을 먼저 생각해 보자.

In [2]:
record Person(String name) {
  public String toString() {
    return "hello " + name.toString(); 
  }
}

You get an exception if you write


In [3]:
System.out.println(new Person(null));

EvalException: Cannot invoke "String.toString()" because "this.name" is null

아래와 같이 매번 검사하는 것은 좀 바보같은 방법이다.

In [4]:
record Person(String name) {
  public String toString() {
    if (name == null) {
      return "hello unnamed"; 
    }
    return "hello " + name.toString(); 
  }
}
System.out.println(new Person(null));


hello unnamed


왜냐하면 `null`인 경우 어떤 값으로 대체해야 할지 의미를 억지로 생각해 내야 하기 때문이다.

그래서 일반적으로 애초에 객체가 생성될 때 멤버가 `null`로 초기화되지 못하도록 하는 것이 낫다.

In [6]:
record Person(String name) {
  public Person {   // it's a compact constructor
    Objects.requireNonNull(name);
  }
  public String toString() {
    return "hello " + name.toString(); 
  }
}

In [7]:
// 생성자를 활용할 때 `null`로 초기화하려면면 예외없이 항상 NullPointerException 발생
new Person(null); 

EvalException: 

한 곳에서 나던 `NullPointerException`을 또 다른 곳에서 발생시킬 뿐인데 뭐가 더 나아졌는가 의문이 들 수 도 있다.

하지만 분명한 차이가 있는데, `NullPointerException`이 `toString()`에서 발생하는 경우는 `null`을 구성요소(또는 멤버)로 포함하는 `Person` 객체가 있더라도 발생할 수도 있고 발생하지 않을 수도 있어 프로그램 사용자의 입장에서는 **무작위로 발생하는 듯**하게 보일 수 있다. 왜냐하면 `toString()`을 모든 객체에 대해 항상 호출하는 것은 아니기 때문이다.

하지만 생성자에서 검사하여 `NullPointerException`을 발생시키는 경우는 `null`로 구성요소를 초기화하려 할 때매다 **예외없이 일관되게** `NullPointerException`이 발생하므로, 문제가 있는지 더 빨리 알아차릴 수 있고 문제의 원인이 무엇인지도 더 쉽게 분석할 가능성이 높아질 것이다.

그러므로 자바에서 불필요한 `NullPointerException`을 피하려면 다음과 같은 방식으로 코드를 작성하라
- 객체의 멤버가 `null`로 초기화되지 못하도록 생성자에서 막아라
- 메소드에서 절대로 `null`을 리턴하지 말라 (`Optional`을 활용하거나 빈 컬렉션을 리턴)

물론 효율적으로 데이터 구조를 구현하기 위해 `null` 참조가 정말 꼭 필요한 경우도 있을 수 있지만,
이는 패키지 내부의 구현으로만 활용하고 외부에 공개하여 드러내지 말아야 한다.

## 방어적 프로그래밍 Defensive programming

외부에서 `null`이 제공되지 않을 것이라는 가정을 하지 말고, 예상치 못하게 `null`이 들어오는 경우에 대해서도 능동적으로 대처하는 코드를 작성하라.
- 

The best way to not store null in a field (or a record component) is to reject any attempt
to call a public method with `null` as argument. So any public methods should call
`Objects.requireNonNull()` on all their arguments that are references.


In [ ]:
record Animal(String kind, boolean wild) {
  public Animal {
    Objects.requireNonNull(kind);
    // no need to do a requireNonNull on 'wild', a boolean can not be null
  }
  public boolean isDangerousWith(Animal animal) {
    Objects.requireNonNull(animal);
    return wild || !kind.equals(animal.kind);
  }
}
new Animal(null, true);


### Map.get()


You may sometimes want to pass null to a public method or return null from a method
but it should be an exceptional case and it should be documented


A good example is `Map.get(key)` that is used a lot and is specified to return `null`
if the key is not stored in the map. refer to use `Map.getOrDefault()` instead


In [ ]:
var map = Map.of("John", 5, "Paul", 7);
System.out.println(map.get("Lena"));
System.out.println(map.getOrDefault("Lena", 0));


See chapter 'list and map' for more information.


## Optional
Optional is a special class which means that a return value of a method may not be present.
Unlike a usual object type that can be null, an Optional can be present or empty.
It forces the user code be prepared to handle an empty Optional.


In the following code a `Car` has a color and optionally has a driver


In [ ]:
public class Car {
  private final Person driver;  // may be null
  private final String color;
  public Car(String color, Person driver) {
    this.color = Objects.requireNonNull(color);
    this.driver = driver;  // may be null
  }
  public String color() {
    return color;
  }
  public Optional<Person> driver() {
    return Optional.ofNullable(driver);
  }
}


Trying to call a method of `Person` on an `Optional`, obviously doesn't work


In [ ]:
var car = new Car("red", null);
var name = car.driver().name(); // doesn't compile


so the user code as to be changed to handle `Optional`, and the fact that
an `Optional` can be empty


In [ ]:
var car = new Car("red", null);
var name = car.driver().map(Person::name).orElse("autopilot");
System.out.println(name);


> Don't use Optional for anything else than a return type
> Never store null in an Optional because it defeats its purpose.
